<a href="https://colab.research.google.com/github/samsiroos/SLM-Architecture/blob/main/%D8%A2%D9%85%D9%88%D8%B2%D8%B4_%D9%88_%D8%AA%D9%88%D9%84%DB%8C%D8%AF_%D9%85%D8%AA%D9%86_%D8%A8%D8%A7_GRU_%D8%AF%D8%B1_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# وارد کردن کتابخانه‌های لازم
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import numpy as np

print("کتابخانه‌ها با موفقیت وارد شدند.")

# --- ۱. آماده‌سازی داده‌ها ---
# متن نمونه برای آموزش مدل
# هرچه متن طولانی‌تر و متنوع‌تر باشد، مدل بهتر عمل می‌کند.
# برای شروع، یک متن فارسی ساده را انتخاب می‌کنیم.
text = """
سلام دوستان عزیز، امیدوارم حالتون خوب باشه.
امروز می‌خواهیم درباره مدل‌های زبانی کوچک صحبت کنیم.
این مدل‌ها برای کاربردهای خاص بسیار مفید هستند.
یادگیری ماشینی دنیای جالبی دارد و هر روز پیشرفت می‌کند.
با تمرین و تکرار می‌توانید در این زمینه متخصص شوید.
"""

# تبدیل تمام متن به حروف کوچک برای یکسان‌سازی
text = text.lower()
print(f"\nمتن اصلی برای آموزش:\n{text}\n")

# --- توکن‌سازی (Tokenization) ---
# توکن‌سازی فرآیند تبدیل متن به دنباله‌ای از اعداد است.
# هر کلمه منحصر به فرد یک عدد یکتا دریافت می‌کند.
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text]) # مدل توکن‌ساز را روی متن آموزش می‌دهیم تا کلمات را بشناسد
word_index = tokenizer.word_index # دیکشنری کلمات به اعداد (ایندکس‌ها)
print(f"فهرست کلمات و ایندکس‌های آن‌ها:\n{word_index}\n")

# تعداد کل کلمات منحصر به فرد + 1 (برای کلماتی که در واژه‌نامه نیستند یا برای padding)
total_words = len(word_index) + 1
print(f"تعداد کل کلمات منحصر به فرد در واژه‌نامه: {total_words}\n")

# --- ساخت دنباله‌های ورودی و خروجی برای آموزش ---
# برای آموزش مدل، دنباله‌هایی از کلمات (n-grams) ایجاد می‌کنیم.
# مثلاً اگر جمله "سلام دوستان عزیز" باشد:
# ورودی: "سلام" -> خروجی: "دوستان"
# ورودی: "سلام دوستان" -> خروجی: "عزیز"

input_sequences = []
for line in text.split('\n'):
    if line.strip() == "": # خطوط خالی را نادیده می‌گیریم
        continue
    token_list = tokenizer.texts_to_sequences([line])[0] # هر خط را به دنباله‌ای از اعداد تبدیل می‌کنیم
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1] # دنباله n-gram (از ابتدای خط تا کلمه فعلی)
        input_sequences.append(n_gram_sequence)

print(f"تعداد دنباله‌های ورودی برای آموزش: {len(input_sequences)}\n")
print(f"نمونه‌ای از دنباله‌های ورودی (اعداد):\n{input_sequences[:5]}\n")

# --- هم‌اندازه‌سازی دنباله‌ها (Padding) و تفکیک X و Y ---
# شبکه‌های عصبی نیاز دارند که ورودی‌ها هم‌اندازه باشند.
# بنابراین، دنباله‌های کوتاه‌تر را با صفر در ابتدا پر می‌کنیم (pre-padding).
max_sequence_len = max([len(x) for x in input_sequences])
print(f"حداکثر طول دنباله در داده‌های آموزشی: {max_sequence_len}\n")

padded_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(input_sequences,
                                                                          maxlen=max_sequence_len,
                                                                          padding='pre'))

# X (ورودی): تمام کلمات دنباله به جز آخرین کلمه
X = padded_sequences[:, :-1]
# labels (خروجی): فقط آخرین کلمه در هر دنباله (کلمه‌ای که می‌خواهیم پیش‌بینی کنیم)
labels = padded_sequences[:, -1]

# تبدیل خروجی (labels) به فرمت one-hot encoding
# چون پیش‌بینی کلمه یک مسئله دسته‌بندی است، خروجی‌ها باید به صورت one-hot باشند.
y = to_categorical(labels, num_classes=total_words)

print(f"ابعاد ورودی (X) برای مدل: {X.shape}") # (تعداد نمونه‌ها, طول دنباله ورودی)
print(f"ابعاد خروجی (y) برای مدل (one-hot): {y.shape}") # (تعداد نمونه‌ها, تعداد کل کلمات در واژه‌نامه)

# --- ۲. ساخت مدل GRU ---
# معماری مدل GRU را تعریف می‌کنیم.
model = Sequential()
# لایه Embedding: کلمات را از اعداد به بردارهای متراکم (dense vectors) تبدیل می‌کند.
# این بردارها معانی کلمات را در خود نگه می‌دارند.
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1)) # total_words: اندازه واژه‌نامه، 100: ابعاد بردار کلمه
# لایه GRU: لایه اصلی شبکه عصبی بازگشتی با دروازه‌ها.
# 150: تعداد واحد حافظه (neurons) در لایه GRU
# return_sequences=False: یعنی فقط خروجی آخرین گام زمانی را برمی‌گرداند.
model.add(GRU(150, return_sequences=False))
# لایه Dense: لایه خروجی با تابع فعال‌سازی softmax.
# total_words: تعداد نورون‌ها برابر با تعداد کل کلمات در واژه‌نامه است.
# softmax: برای تبدیل خروجی به توزیع احتمال روی کلمات.
model.add(Dense(total_words, activation='softmax'))

# کامپایل کردن مدل: تعریف تابع هزینه، بهینه‌ساز و معیار ارزیابی
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

# --- ۳. آموزش مدل ---
# این مرحله ممکن است کمی طول بکشد. در Google Colab می‌توانید از GPU استفاده کنید
# (Runtime -> Change runtime type -> Hardware accelerator -> GPU)
# epochs: تعداد دوره‌های آموزش (مدل چند بار کل داده‌ها را ببیند و یاد بگیرد).
# verbose=1: برای نمایش جزئیات آموزش در هر دوره.
print("\nشروع آموزش مدل GRU...")
history = model.fit(X, y, epochs=100, verbose=1) # می‌توانید تعداد epochs را بیشتر کنید

print("\nآموزش مدل به پایان رسید.")
print(f"دقت نهایی مدل در آموزش: {history.history['accuracy'][-1]:.4f}")
print(f"خطای نهایی مدل در آموزش: {history.history['loss'][-1]:.4f}")

# --- ۴. تولید متن با مدل آموزش‌دیده ---
# تابعی برای تولید کلمات بعدی بر اساس یک متن اولیه (seed text)
def generate_text(seed_text, next_words, model, max_sequence_len, tokenizer):
    generated_text = seed_text
    for _ in range(next_words):
        # تبدیل متن اولیه به دنباله عددی
        token_list = tokenizer.texts_to_sequences([generated_text])[0]
        # هم‌اندازه‌سازی دنباله برای ورودی مدل
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list],
                                                                  maxlen=max_sequence_len-1,
                                                                  padding='pre')

        # پیش‌بینی احتمال هر کلمه به عنوان کلمه بعدی
        predicted_probs = model.predict(token_list, verbose=0)[0]
        # انتخاب کلمه‌ای با بالاترین احتمال
        predicted_word_index = np.argmax(predicted_probs)

        output_word = ""
        # پیدا کردن کلمه متناظر با ایندکس پیش‌بینی شده
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break

        # اگر کلمه‌ای پیدا نشد یا مدل کلمه نامعلومی پیش‌بینی کرد
        if output_word == "":
            output_word = "<unk>" # برای کلمات ناشناخته

        generated_text += " " + output_word
    return generated_text

# --- امتحان تولید متن ---
# یک کلمه یا عبارت اولیه بدهید تا مدل از آنجا شروع به تولید متن کند.
seed_text_input = "مدل‌های زبانی" # می‌توانید این را تغییر دهید، مثلاً "سلام" یا "یادگیری"
num_words_to_generate = 15 # تعداد کلماتی که می‌خواهید مدل تولید کند

print(f"\nشروع تولید متن با عبارت اولیه: '{seed_text_input}'")
generated_sentence = generate_text(seed_text_input, num_words_to_generate, model, max_sequence_len, tokenizer)
print(f"\nمتن تولید شده توسط مدل:\n{generated_sentence}")

# مثال دیگر
seed_text_input_2 = "دنیای ماشینی"
generated_sentence_2 = generate_text(seed_text_input_2, num_words_to_generate, model, max_sequence_len, tokenizer)
print(f"\nمتن تولید شده دیگر توسط مدل:\n{generated_sentence_2}")

کتابخانه‌ها با موفقیت وارد شدند.

متن اصلی برای آموزش:

سلام دوستان عزیز، امیدوارم حالتون خوب باشه.
امروز می‌خواهیم درباره مدل‌های زبانی کوچک صحبت کنیم.
این مدل‌ها برای کاربردهای خاص بسیار مفید هستند.
یادگیری ماشینی دنیای جالبی دارد و هر روز پیشرفت می‌کند.
با تمرین و تکرار می‌توانید در این زمینه متخصص شوید.


فهرست کلمات و ایندکس‌های آن‌ها:
{'این': 1, 'و': 2, 'سلام': 3, 'دوستان': 4, 'عزیز،': 5, 'امیدوارم': 6, 'حالتون': 7, 'خوب': 8, 'باشه': 9, 'امروز': 10, 'می\u200cخواهیم': 11, 'درباره': 12, 'مدل\u200cهای': 13, 'زبانی': 14, 'کوچک': 15, 'صحبت': 16, 'کنیم': 17, 'مدل\u200cها': 18, 'برای': 19, 'کاربردهای': 20, 'خاص': 21, 'بسیار': 22, 'مفید': 23, 'هستند': 24, 'یادگیری': 25, 'ماشینی': 26, 'دنیای': 27, 'جالبی': 28, 'دارد': 29, 'هر': 30, 'روز': 31, 'پیشرفت': 32, 'می\u200cکند': 33, 'با': 34, 'تمرین': 35, 'تکرار': 36, 'می\u200cتوانید': 37, 'در': 38, 'زمینه': 39, 'متخصص': 40, 'شوید': 41}

تعداد کل کلمات منحصر به فرد در واژه‌نامه: 42

تعداد دنباله‌های ورودی برای آموزش: 38

نمونه‌ای از دنباله‌های ور

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


شروع آموزش مدل GRU...
Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.0280 - loss: 3.7358
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.2204 - loss: 3.7176 
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.3076 - loss: 3.7038 
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.3602 - loss: 3.6906
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3427 - loss: 3.6769
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4057 - loss: 3.6627
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4265 - loss: 3.6461
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3810 - loss: 3.6280
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3986 - loss: 3.6080
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3602 - loss: 3.5824
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.3147 - loss: 3.5588
Epoch 12/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/ste